In [1]:
import math
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("iris.csv", header=None)
#shuffle data
ds = df.sample(frac=1)
#split data into 80% train, 20% test
train, test = train_test_split(ds,test_size=0.2)
train_att = train[[0,1,2,3]]
test_att = test[[0,1,2,3]]
#normalize data
train_norm = (train_att - train_att.mean()) / train_att.std()
test_norm = (test_att - train_att.mean()) / train_att.std()
train_norm.fillna(0, inplace=True)
test_norm.fillna(0, inplace=True)

In [11]:
'''
k - number of neighbors
data - dataset
instance - instance to classify
'''
def knn(k,data,instance):
    distances = data.apply(lambda row: math.dist(row,instance), axis=1)
    #get k closest instances (including the input instance)
    k_neighbors = distances.sort_values()[:k]
    #get class value with largest number of occurences
    predicted_class = df.loc[k_neighbors.index,[4]][4].mode()
    return predicted_class[0]

knn(5,train_norm,train_norm.iloc[1])

'Iris-virginica'

In [13]:
'''
train knn using training data
return the accuracy of the model
'''
def train_knn(k,data):
    predictions = data.apply(lambda row: knn(k,data,row), axis=1)
    actual_labels = df.loc[predictions.index,[4]]
    return predictions.eq(actual_labels[4].values).mean()

train_knn(5,train_norm)

0.9666666666666667